In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
import mediapipe as mp

In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [6]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_DATA_latest/HELLO') 

# Actions that we try to detect
actions = np.array(['hello', 'not hello'])

# videos worth of data
no_sequences = 1000

# Videos are going to be 30 frames in length
sequence_length = 23

label_map = {label:num for num, label in enumerate(actions)}


In [7]:
label_map


{'hello': 0, 'not hello': 1}

In [8]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [9]:
X = np.array(sequences)


In [10]:
X.shape


(2000, 23, 1662)

In [11]:
y = to_categorical(labels).astype(int)


In [12]:
y

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)


In [14]:
y_test.shape


(100, 2)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(23,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [18]:
res = [.7, 0.2, 0.1]


In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [30]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])


Epoch 1/100
60/60 [==============================] - 11s 155ms/step - loss: 0.1543 - categorical_accuracy: 0.9579
Epoch 2/100
60/60 [==============================] - 9s 147ms/step - loss: 0.1451 - categorical_accuracy: 0.9589
Epoch 3/100
60/60 [==============================] - 9s 150ms/step - loss: 0.1468 - categorical_accuracy: 0.9589
Epoch 4/100
60/60 [==============================] - 9s 155ms/step - loss: 0.1361 - categorical_accuracy: 0.9616
Epoch 5/100
60/60 [==============================] - 9s 146ms/step - loss: 0.1718 - categorical_accuracy: 0.9553
Epoch 6/100
 8/60 [===>..........................] - ETA: 7s - loss: 0.1351 - categorical_accuracy: 0.9648

KeyboardInterrupt: 

In [31]:
res = model.predict(X_test)


4/4 [==============================] - 0s 41ms/step


In [33]:
model.save('hello.h5')


In [34]:
model.load_weights('hello.h5')


In [35]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score


In [36]:
yhat = model.predict(X_test)


4/4 [==============================] - 0s 48ms/step


In [37]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [38]:
accuracy_score(ytrue, yhat)


0.96